# Add web search ability to research assistant

In [1]:
! pip install "pyautogen>=0.2.18" "tavily-python"


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [8]:
## import required libraries
import os
from typing import Annotated
from tavily import TavilyClient
from autogen import AssistantAgent, UserProxyAgent, config_list_from_json, register_function
from dotenv import load_dotenv

load_dotenv() # load environment variables from .env file

True

In [5]:
## llm configuration
llm_config ={
                "model": "gpt-4o",
                "api_key": os.getenv("OPENAI_API_KEY")
            }

## Tavily as a tool for searching the web

In [9]:
tavily = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))

def search_tool(query: Annotated[str, "The search query"]) -> Annotated[str, "The search results"]:
    return tavily.get_search_context(query=query, search_depth="advanced")

## Adding a ReAct prompt

In [10]:
ReAct_prompt = """
Answer the following questions as best you can. You have access to tools provided.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take
Action Input: the input to the action
Observation: the result of the action
... (this process can repeat multiple times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!
Question: {input}
"""

# Define the ReAct prompt message. Assuming a "question" field is present in the context


def react_prompt_message(sender, recipient, context):
    return ReAct_prompt.format(input=context["question"])

## Define agents and set up code executors

In [15]:
user_proxy = UserProxyAgent(
    name="User",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    human_input_mode="ALWAYS",
    max_consecutive_auto_reply=10,
    code_execution_config=False
)

research_assistant = AssistantAgent(
    name="Assistant",
    system_message="""
                You are a helpful research assistant who has the ability to search the web using the provided tools.
                Only use the tools you have been provided with. Reply TERMINATE when the task is done.
                """,
    llm_config=llm_config,
)



[autogen.oai.client: 09-03 00:54:00] {164} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


## Register the search tool

In [17]:
# Register the search tool.
register_function(
    search_tool,
    caller=research_assistant,
    executor=user_proxy,
    name="search_tool",
    description="Search the web for the given query",
)


[autogen.oai.client: 09-03 00:54:15] {164} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


## Initiate the chat

In [ ]:
user_proxy.initiate_chat(
    research_assistant,
    message=react_prompt_message,
    question="Who won the T20 cricket world cup in 2024?",
)

User (to Assistant):


Answer the following questions as best you can. You have access to tools provided.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take
Action Input: the input to the action
Observation: the result of the action
... (this process can repeat multiple times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!
Question: Who won the T20 cricket world cup in 2024?


--------------------------------------------------------------------------------
Assistant (to User):

***** Suggested tool call (call_KzaKQDT41HqEEX1QSYVItNFN): search_tool *****
Arguments: 
{"query":"T20 cricket world cup winner 2024"}
****************************************************************************

--------------------------------------------------------------------------------


Replying as User. Provide feedback to Assistant. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  



>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING FUNCTION search_tool...
User (to Assistant):

User (to Assistant):

***** Response from calling tool (call_KzaKQDT41HqEEX1QSYVItNFN) *****
"[\"{\\\"url\\\": \\\"https://en.wikipedia.org/wiki/2024_ICC_Men's_T20_World_Cup_final\\\", \\\"content\\\": \\\"The 2024 ICC Men's T20 World Cup final was a Twenty20 International cricket match played at Kensington Oval in Bridgetown, Barbados on 29 June 2024 to determine the winner of the 2024 ICC Men's T20 World Cup. [1] [2] It was played between South Africa and India.[3]India defeated South Africa by 7 runs to win their second T20 World Cup title. [4] Virat Kohli was named Player of the Match for ...\\\"}\", \"{\\\"url\\\": \\\"https://www.icc-cricket.com/tournaments/t20cricketworldcup/videos/india-script-stunning-title-win-match-highlights-sa-v-ind-t20wc-2024-final\\\", \\\"content\\\": \\\"ICC Men's T20 World Cup, 2024. India script stunning title win | Match